In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression, Lasso, RidgeCV, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector as selector
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import set_config
from xgboost import XGBRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import ParameterGrid
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR
from sklearn.metrics import make_scorer
from sklearn.kernel_ridge import KernelRidge

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
set_config(display='diagram')
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12,8)

In [ ]:
trainData = 'train.csv'
testData = 'test.csv'
index_col = 'Id'
target_col = 'SalePrice'

In [ ]:
def cleanData(X):
    """ Handle outlier remove outside of Pipeline """
    X = X[X['GrLivArea'] < 4000]
    X = X.loc[X.index.difference([30, 88, 462, 631, 1322])]
    return X

In [ ]:
class processHousePrices:
    
    def __init__(self):
        self._prepared = False
        # Store each feature in a defined group
        # Define encoding of ordinal categories to numeric
        self._ordinalEncoding = ({
            'Alley'       : {'None': 0, 'Grvl': 1, 'Pave': 2},
            'BsmtCond'    : {'No': 0, 'Po': 1, 'Fa': 2, 
                             'TA': 3, 'Gd': 4, 'Ex': 5},
            'BsmtExposure': {'No': 0, 'Mn': 1, 'Av': 2, 'Gd': 3},
            'BsmtFinType1': {'No': 0, 'Unf': 1, 'LwQ': 2, 'Rec' : 3, 
                             'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
            'BsmtFinType2': {'No': 0, 'Unf': 1, 'LwQ': 2, 'Rec' : 3, 
                             'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
            'BsmtQual'    : {'No': 0, 'Po': 1, 'Fa': 2, 
                             'TA': 3, 'Gd': 4, 'Ex': 5},
            'ExterCond'   : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
            'ExterQual'   : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
            'FireplaceQu' : {'No': 0, 'Po': 1, 'Fa': 2, 
                             'TA': 3, 'Gd': 4, 'Ex': 5},
            'Functional'  : {'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1' : 4, 
                             'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8},
            'GarageCond'  : {'No': 0, 'Po': 1, 'Fa': 2, 
                             'TA': 3, 'Gd': 4, 'Ex': 5},
            'GarageQual'  : {'No': 0, 'Po': 1, 'Fa': 2, 
                             'TA': 3, 'Gd': 4, 'Ex': 5},
            'HeatingQC'   : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
            'KitchenQual' : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
            'LandSlope'   : {'Sev': 1, 'Mod': 2, 'Gtl': 3},
            'LotShape'    : {'IR3': 1, 'IR2': 2, 'IR1': 3, 'Reg' : 4},
            'PavedDrive'  : {'N': 0, 'P': 1, 'Y': 2},
            'PoolQC'      : {'No': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
            'Street'      : {'Grvl': 1, 'Pave': 2},
            'Utilities'   : {'ELO': 1, 'NoSeWa': 2, 
                             'NoSewr':  3, 'AllPub': 4},
            'Fence'       : {'No': 0, 'MnWw': 1, 'GdWo': 2, 
                             'MnPrv': 3, 'GdPrv': 4},
            'GarageFinish': {'No': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
            'OverallQual' : {},
            'OverallCond' : {},
        })

        
    def _allFeatures(self):
        """ Return all values in features """
        return [ft for group in list(self._features.values()) for ft in group]

    
    def _assertValidFeatures(self):
        """ Assert feature dictionary matches the input """
        assert sorted(self._allFeatures()) == sorted(list(self._X.columns))
        
        
    def _remapOrdinal(self):
        """ Perform remapping of ordinal strings to numeric """
        self._X = self._X.replace(self._ordinalEncoding)
        
       
    def _addFeature(self, name, category, value):
        """ Add feature to data and update feature dictionary """
        self._X[name] = value
        self._addFeatureDict(name, category)
        
        
    def _addFeatureDict(self, featureName, group):
        assert featureName not in self._features.values()
        try:
            self._features[group].append(featureName)
        except KeyError:
            self._features[group] = [featureName]
    
    
    def _removeFeatures(self, name):
        self._X = self._X.drop(name, axis=1)
        self._removeFeatureDict(name)
        
        
    def _removeFeatureDict(self, name):
        for category, feature in self._features.items():
            try:
                feature.remove(name)
                break
            except ValueError:
                pass
        
        
    def _categoriesToString(self):
        """ Ensure all categories are string. Add prefix to prevent
            casting back to numeric. """
        for feature in self._features['categorical']:
            self._X[feature] = 'str' + self._X[feature].astype(str)
    
    
    def getFeatures(self, asIndex=False):
        """ Return feature dictionary with updated categories """
        assert self._prepared, 'Must called prepData() method first'
        features = self._features.copy()
        if asIndex:
            for category, values in features.items():
                features[category] = np.nonzero(self._X.columns.isin(values))[0]
        return features
            
        
    def _setFeatures(self):
        """ Set default assignment of features """
        self._features = ({
            'categorical': ([
                'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',  
                'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 
                'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                'MasVnrType', 'Foundation', 'Heating', 'Electrical', 
                'GarageType', 'GarageYrBlt', 'YearBuilt', 'MiscFeature', 
                'YrSold', 'SaleType', 'SaleCondition', 
                'YearRemodAdd']),
            'ordinal': ([
                'LandSlope', 'OverallQual', 'ExterQual', 'ExterCond', 
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 
                'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 
                'PavedDrive', 'PoolQC', 'Fence', 'OverallCond', 'MoSold']),
            'count': ([
                'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 
                'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
                'TotRmsAbvGrd', 'Fireplaces', 'GarageCars']),
            'realMult': ([
                'LotFrontage', 'LotArea', 'MasVnrArea',
                'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                'LowQualFinSF', 'GrLivArea', 'GarageArea',
                'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
                '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']),
            'binary': ([
                'CentralAir']),
            'realAdd': [],
            'binomial': []
            })
        
    
    def _engineerFeatures(self):
        self._addFeature('hasPool', 'binary', (self._X['PoolArea'] > 0) * 1)
        self._addFeature('has2ndfloor', 'binary', (self._X['2ndFlrSF'] > 0) * 1)
        self._addFeature('hasgarage', 'binary', (self._X['GarageArea'] > 0) * 1)
        self._addFeature('hasbsmt', 'binary', (self._X['TotalBsmtSF'] > 0) * 1)
        self._addFeature('hasfireplace', 'binary', (self._X['Fireplaces'] > 0) * 1)
        self._addFeature('YrBltAndRemod', 'categorical', 
                         self._X['YearBuilt'] + self._X['YearRemodAdd'])
        self._addFeature('TotalSF', 'realMult', 
                         (self._X['TotalBsmtSF'] + self._X['1stFlrSF'] 
                          + self._X['2ndFlrSF']))
        self._addFeature('Total_sqr_footage', 'realMult', 
                         (self._X['BsmtFinSF1'] + self._X['BsmtFinSF2']
                         + self._X['1stFlrSF'] + self._X['2ndFlrSF']))
        self._addFeature('Total_HalfBath', 'count', 
                         (2 * (self._X['FullBath'] + self._X['BsmtFullBath']) 
                          + self._X['HalfBath'] + self._X['BsmtHalfBath']))
        self._addFeature('Total_porch_sf', 'realMult',
                       (self._X['OpenPorchSF'] + self._X['3SsnPorch'] 
                        + self._X['EnclosedPorch'] + self._X['ScreenPorch']
                        + self._X['WoodDeckSF']))
        self._addFeature('MonthSold_sin', 'realMultNorm',
                        np.sin(2 * np.pi * (self._X['MoSold'] / self._X['MoSold'].max())))
        self._addFeature('MonthSold_cos', 'realMultNorm',
                        np.cos(2 * np.pi * (self._X['MoSold'] / self._X['MoSold'].max())))
        #self._addFeature('hasFence', 'binary', (self._X['Fence'] > 0) * 1)
        self._addFeature('MSZoning2', 'categorical', 
                         self._X['MSZoning'].replace({'RH' : 'R', 'RL': 'R', 'RP': 'R', 'RM': 'R'}))
        #self._addFeature('beenRemodelled', 'binary',
        #                 (self._X['YearBuilt'] != self._X['YearRemodAdd']) * 1)
        
        
    def _dropFeatures(self):
        self._removeFeatures('PoolQC')
        self._removeFeatures('Utilities')
        self._removeFeatures('Street')
        self._removeFeatures('MoSold')
        #self._removeFeatures('MSZoning')
          
            
    def prepData(self, data):
        self._X = data
        self._setFeatures()
        self._remapOrdinal()
        self._engineerFeatures()
        self._dropFeatures()
        self._categoriesToString()
        self._assertValidFeatures()
        self._prepared = True
        return self._X

In [ ]:
# Initiliase pre-model data processing
processor = processHousePrices()

In [ ]:
X = cleanData(pd.read_csv(trainData, index_col='Id'))
y = X.pop('SalePrice')
X = processor.prepData(X)
features = processor.getFeatures(asIndex=True)

In [ ]:
split = train_test_split(X, y, random_state=0, train_size=0.8, test_size=0.2)
X_train, X_valid, y_train, y_valid = map(lambda x: x.copy(), split)

In [ ]:
realMultTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler',  RobustScaler()),
    ('power',   PowerTransformer(method='yeo-johnson'))])
countTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler',  RobustScaler())])
categoryTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('onehot' , OneHotEncoder(handle_unknown='ignore'))])
ordinalTransfomer = Pipeline(steps=[
    ('imputer', SimpleImputer())])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('realMult', realMultTransformer, features['realMult']),
    ('count',    countTransformer,    features['count']),
    ('category', categoryTransformer, np.concatenate((features['categorical'], features['binary']))),
    ('ordinal',  ordinalTransfomer,   features['ordinal'])],
    remainder='passthrough')

In [ ]:
# We need to run a grid search of optimal parameters for each model seperately.
# Then use these parameters for the grid search
# Ideally you could wrap the whole thing as a grid seach but will be VERY slow

In [ ]:
# Configure the cross-validation procedure
cv = KFold(n_splits=5, shuffle=True, random_state=1)

In [ ]:
# Define custom score based on Kaggle competition
def kaggleLoss(y_true, y_pred):
    return mean_squared_error(np.log(y_true), np.log(y_pred), squared=False)
kaggleScore = make_scorer(kaggleLoss, greater_is_better=False)

In [ ]:
models = [
    {'regressor': XGBRegressor(n_jobs=12, random_state=0),
     'preprocessor': preprocessor,
     'params': 
         {'preprocessor__realMult__imputer__strategy': Categorical(['median']),
          'preprocessor__count__imputer__strategy':    Categorical(['median']),
          'preprocessor__category__imputer__strategy': Categorical(['constant']),
          'preprocessor__ordinal__imputer__strategy':  Categorical(['most_frequent']),
          'classifier__regressor__n_estimators': Integer(100, 1000),
          'classifier__regressor__max_depth': Integer(3, 12),
          'classifier__regressor__learning_rate': Real(0.001, 0.1, prior='log-uniform')
         }
    },
    {'regressor': Lasso(max_iter=1e7, random_state=0),
     'preprocessor': preprocessor,
     'params':
         {'preprocessor__realMult__imputer__strategy': Categorical(['median']),
          'preprocessor__count__imputer__strategy':    Categorical(['median']),
          'preprocessor__category__imputer__strategy': Categorical(['constant']),
          'preprocessor__ordinal__imputer__strategy':  Categorical(['most_frequent']),
          'classifier__regressor__alpha': Real(1e-6, 0.0008, prior='log-uniform')
         }
    },
    {'regressor': SVR(),
     'preprocessor': preprocessor,
     'params':
         {'preprocessor__realMult__imputer__strategy': Categorical(['median']),
          'preprocessor__count__imputer__strategy': Categorical(['median']),
          'preprocessor__category__imputer__strategy': Categorical(['constant']),
          'preprocessor__ordinal__imputer__strategy': Categorical(['most_frequent']),
          'classifier__regressor__C': Integer(1, 50),
          'classifier__regressor__epsilon': Real(1e-6, 1, prior='log-uniform'),
          'classifier__regressor__gamma': Categorical(['auto', 'scale']),
          'classifier__regressor__kernel': ['rbf']
         }
    },
    {'regressor': ElasticNet(),
     'preprocessor': preprocessor,
     'params':
         {'preprocessor__realMult__imputer__strategy': Categorical(['median']),
          'preprocessor__count__imputer__strategy':    Categorical(['median']),
          'preprocessor__category__imputer__strategy': Categorical(['constant']),
          'preprocessor__ordinal__imputer__strategy':  Categorical(['most_frequent']),
          'classifier__regressor__alpha': Real(1e-6, 1, prior='log-uniform'), 
          'classifier__regressor__l1_ratio': Real(1e-6, 1, prior='log-uniform'),
         }
    },
    {'regressor': KernelRidge(),
     'preprocessor': preprocessor,
     'params':
         {'preprocessor__realMult__imputer__strategy': Categorical(['median']),
          'preprocessor__count__imputer__strategy':    Categorical(['median']),
          'preprocessor__category__imputer__strategy': Categorical(['constant']),
          'preprocessor__ordinal__imputer__strategy':  Categorical(['most_frequent']),
          'classifier__regressor__alpha': Real(1e-6, 1, prior='log-uniform'), 
        }
    }
]

In [ ]:
estimators = []
stackBestParams = {}
for model in models[1:]:
    regressor = TransformedTargetRegressor(
        regressor=model['regressor'], 
        func=np.log1p, inverse_func=np.expm1)
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', regressor)])
    gridSearch = BayesSearchCV(
        pipeline, model['params'], 
        scoring=kaggleScore, cv=cv, refit=True, verbose=2)
    gridSearch.fit(X_train, y_train)
    #gridSearch.fit(X, y)
    name = model["regressor"].__class__.__name__
    # Add regressor best params
    for param, value in gridSearch.best_params_.items():
        stackBestParams[f'{name}__{param}'] = [value]
    # Add pipeline for stackRegressor
    estimators.append((name, pipeline))
    
    predictions = gridSearch.predict(X_valid)
    score = kaggleLoss(y_valid, predictions)
    fig, ax = plt.subplots()
    ax.plot([y_valid.min(), y_valid.max()],
            [y_valid.min(), y_valid.max()],
            '--r', linewidth=2, alpha=0.75)
    sns.scatterplot(x=y_valid, y=predictions, ax=ax)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    ax.set_title(f'{name} (score = {score:.4f})', loc='left')
    fig.tight_layout()
    fig.savefig(f'plots/{name}.png')

In [ ]:
stackingRegressor = StackingRegressor(estimators=estimators)
stackingGridSearch = GridSearchCV(
    stackingRegressor, stackBestParams, 
    scoring=kaggleScore, cv=cv, refit=True)
stackingGridSearch.fit(X_train, y_train)
#stackingGridSearch.fit(X, y)

In [ ]:
name = stackingRegressor.__class__.__name__
predictions = stackingGridSearch.predict(X_valid)
score = kaggleLoss(y_valid, predictions)
fig, ax = plt.subplots()
ax.plot([y_valid.min(), y_valid.max()],
        [y_valid.min(), y_valid.max()],
        '--r', linewidth=2, alpha=0.75)
sns.scatterplot(x=y_valid, y=predictions, ax=ax)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
ax.set_title(f'{name} (score = {score:.4f})', loc='left')
fig.tight_layout()

In [ ]:
X_test = pd.read_csv(testData, index_col='Id')
# Initiliase pre-model data processing

X_test = processor.prepData(X_test)
preds = gridSearch.predict(X_test)
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds})
assert output.shape == (1459, 2)
output.to_csv('submission.csv', index=False)

In [ ]:
    if False:
        X['Alley'] = X['Alley'].fillna('None')
        # BedroomAbvGr : NA most likely means 0
        X['BedroomAbvGr'] = X['BedroomAbvGr'].fillna(0)
        # BsmtQual etc : data description says NA for basement features is 'no basement'
        X['BsmtQual'] = X['BsmtQual'].fillna('No')
        X['BsmtCond'] = X['BsmtCond'].fillna('No')
        X['BsmtExposure'] = X['BsmtExposure'].fillna('No')
        X['BsmtFinType1'] = X['BsmtFinType1'].fillna('No')
        X['BsmtFinType2'] = X['BsmtFinType2'].fillna('No')
        X['BsmtFullBath'] = X['BsmtFullBath'].fillna(0)
        X['BsmtHalfBath'] = X['BsmtHalfBath'].fillna(0)
        X['BsmtUnfSF'] = X['BsmtUnfSF'].fillna(0)
        # CentralAir : NA most likely means No
        X['CentralAir'] = X['CentralAir'].fillna('N')
        # Condition : NA most likely means Normal
        X['Condition1'] = X['Condition1'].fillna('Norm')
        X['Condition2'] = X['Condition2'].fillna('Norm')
        # EnclosedPorch : NA most likely means no enclosed porch
        X['EnclosedPorch'] = X['EnclosedPorch'].fillna(0)
        # External stuff : NA most likely means average
        X['ExterCond'] = X['ExterCond'].fillna('TA')
        X['ExterQual'] = X['ExterQual'].fillna('TA')
        # Fence : data description says NA means 'no fence'
        X['Fence'] = X['Fence'].fillna('No')
        # FireplaceQu : data description says NA means 'no fireplace'
        X['FireplaceQu'] = X['FireplaceQu'].fillna('No')
        X['Fireplaces'] = X['Fireplaces'].fillna(0)
        # Functional : data description says NA means typical
        X['Functional'] = X['Functional'].fillna('Typ')
        # GarageType etc : data description says NA for garage features is 'no garage'
        X['GarageType'] = X['GarageType'].fillna('No')
        X['GarageFinish'] = X['GarageFinish'].fillna('No')
        X['GarageQual'] = X['GarageQual'].fillna('No')
        X['GarageCond'] = X['GarageCond'].fillna('No')
        X['GarageArea'] = X['GarageArea'].fillna(0)
        X['GarageCars'] = X['GarageCars'].fillna(0)
        # HalfBath : NA most likely means no half baths above grade
        X['HalfBath'] = X['HalfBath'].fillna(0)
        # HeatingQC : NA most likely means typical
        X['HeatingQC'] = X['HeatingQC'].fillna('TA')
        # KitchenAbvGr : NA most likely means 0
        X['KitchenAbvGr'] = X['KitchenAbvGr'].fillna(0)
        # KitchenQual : NA most likely means typical
        X['KitchenQual'] = X['KitchenQual'].fillna('TA')
        # LotFrontage : NA most likely means no lot frontage
        X['LotFrontage'] = X['LotFrontage'].fillna(0)
        # LotShape : NA most likely means regular
        X['LotShape'] = X['LotShape'].fillna('Reg')
        # MasVnrType : NA most likely means no veneer
        X['MasVnrType'] = X['MasVnrType'].fillna('None')
        X['MasVnrArea'] = X['MasVnrArea'].fillna(0)
        # MiscFeature : data description says NA means 'no misc feature'
        X['MiscFeature'] = X['MiscFeature'].fillna('No')
        X['MiscVal'] = X['MiscVal'].fillna(0)
        # OpenPorchSF : NA most likely means no open porch
        X['OpenPorchSF'] = X['OpenPorchSF'].fillna(0)
        # PavedDrive : NA most likely means not paved
        X['PavedDrive'] = X['PavedDrive'].fillna('N')
        # PoolQC : data description says NA means 'no pool'
        X['PoolQC'] = X['PoolQC'].fillna('No')
        X['PoolArea'] = X['PoolArea'].fillna(0)
        # SaleCondition : NA most likely means normal sale
        X['SaleCondition'] = X['SaleCondition'].fillna('Normal')
        # ScreenPorch : NA most likely means no screen porch
        X['ScreenPorch'] = X['ScreenPorch'].fillna(0)
        # TotRmsAbvGrd : NA most likely means 0
        X['TotRmsAbvGrd'] = X['TotRmsAbvGrd'].fillna(0)
        # Utilities : NA most likely means all public utilities
        X['Utilities'] = X['Utilities'].fillna('AllPub')
        # WoodDeckSF : NA most likely means no wood deck
        X['WoodDeckSF'] = X['WoodDeckSF'].fillna(0)

In [ ]:
for a in stackingRegressor.get_params().keys():
    print(a)